A Genetic/Memetic Algorithm for TSP problem.

AhmadReza Nopoush - id:610301194

we use random library to produce random population for GA and MA.

In [1]:
import random

we define Traverse class as Gene in GA. 
object Travers contains attributes like: 

1. size: represents the count of the cities/points.

2. travel: represents a permutation of the number {1,2,...,n}. each number represent a city/point.

3. fitness: It represents the entire path resulting from traversing the permutation


In [2]:
class Traverse:
    def __init__(self,count:int,cities:dict) -> None:
        self.size = count
        self.travel = list(range(1,1+count))
        random.shuffle(self.travel)
        self.fitness = self.totalPath(cities)
        
    def path(self, city1, city2, cities:dict) -> float:
        distance = 0
        distance += ((cities[city1][0] - cities[city2][0])**2)  
        distance += ((cities[city1][1] - cities[city2][1])**2) 
        return distance**0.5
        
    def totalPath(self, cities:dict) -> float:
        distance = 0
        for i in range(self.size-1):
            distance += self.path(self.travel[i],self.travel[i+1],cities)
        distance += self.path(self.travel[0],self.travel[self.size-1],cities)
        return distance*0.5

the "totalPath" is our fitness function. it recieve dictionaray which the keys are city number and the value is its coordinates.

The lower the fitness, the better the generated traverse. so we should minimize the totalPath function.

the "Mutation" function change the traverse randomly.

With probability alpha, a sublist of the travel list is reversed.

sublist is chosen randomly

In [3]:
def Mutation(gene:Traverse,alpha:float,Cities:dict) -> Traverse:
        if random.random() < alpha:
            a = random.randint(0,gene.size-1)
            b = random.randint(a,gene.size-1)
            reversed = gene.travel[a:b]
            reversed.reverse()
            gene.travel = gene.travel[:a] + reversed + gene.travel[b:]
            gene.fitness = gene.totalPath(Cities)
        return gene

The "CrossOver" function is a function that produces a new answer to the problem. This function receives two traverse at the input and crosses them with the PMX method. PMX method is chosen because the produced child has the most sequence similarity to its parents.

In [4]:
def CrossOver(dad : Traverse, mom : Traverse, Cities:dict) -> Traverse:
    child = Traverse(dad.size,Cities)
    randidx = random.randint(2,mom.size+1)
    child.travel = mom.travel[:randidx]
    for i in range(randidx,dad.size):
        if dad.travel[i] not in child.travel:
            child.travel.append(dad.travel[i])
        else:
            j = i
            while dad.travel[j] in child.travel:
                j = mom.travel.index(dad.travel[j])
            child.travel.append(dad.travel[j])
    child.fitness = child.totalPath(Cities)
    return child

finally, The Genetic Algorithm: first we describe the variables->

1. iteration: it shows the number of generation.

2. Cities: Represents a dictionary that keys represent the city number and the value of each key contains a two-member list that represents the coordinates of each city.

3. alpha: Probability of mutation in each crossover

4. Citycount: count of cities

5. Population: Number of Traverses per generation

----------

Number of crossovers per generation: 25% population

Choosing a parent: Rank-based Method


In [5]:
def GA(iteration:int, Cities:dict, alpha:float, citycount:int, Population:int) -> Traverse:
    population = [Traverse(citycount,Cities) for _ in range(Population)]
    population.sort(key = lambda x : x.fitness)
    
    for _ in range(iteration):
        children = []
        for t in range(Population//4):
            parents = random.choices(population,k = 2,weights = [1/(gene+1) for gene in range(1,len(population)+1)])
            child = CrossOver(parents[0],parents[1],Cities)
            child = Mutation(child,alpha,Cities)
            children.append(child)
        
        population += children
        population.sort(key = lambda x : x.fitness)
        population = population[:Population]
        
        print(_,'best_fitness:' , population[0].fitness)
    return population[0]

Now, We can test our code!

convert your input to txt file and put it in this folder.

In [16]:
Cities = dict()
with open("pr1002.txt",mode="r") as file:
    count = 0
    for line in file.readlines():
        line = str(line)
        line = line[:-1]
        line = line.split(" ")
        if count == 0:
            CityCount = int(line[0])
            count = 1
        else:
            Cities.update({int(line[0]):[float(line[1]),float(line[2])]})
best_ga = GA(7000,Cities,0.5,CityCount,30)
print(best_ga.travel)

0 best_fitness: 3120157.02176529
1 best_fitness: 3120157.02176529
2 best_fitness: 3120157.02176529
3 best_fitness: 3120157.02176529
4 best_fitness: 3120157.02176529
5 best_fitness: 3120157.02176529
6 best_fitness: 3111002.0064058695
7 best_fitness: 3111002.0064058695
8 best_fitness: 3111002.0064058695
9 best_fitness: 3111002.0064058695
10 best_fitness: 3111002.0064058695
11 best_fitness: 3111002.0064058695
12 best_fitness: 3109488.294682283
13 best_fitness: 3109488.294682283
14 best_fitness: 3108235.8980553197
15 best_fitness: 3108235.8980553197
16 best_fitness: 3105510.5043128873
17 best_fitness: 3105445.8515944094
18 best_fitness: 3105445.8515944094
19 best_fitness: 3105445.8515944094
20 best_fitness: 3098564.1594522926
21 best_fitness: 3094757.105358582
22 best_fitness: 3094757.105358582
23 best_fitness: 3094757.105358582
24 best_fitness: 3083516.973546642
25 best_fitness: 3083516.973546642
26 best_fitness: 3077184.604661253
27 best_fitness: 3077184.604661253
28 best_fitness: 307718

Result:

1. from gr229: Within 2000 generations and population = 40, we will reach the number 1939 by spending 17 seconds. we tried it several times and the number was between 1900 to 2000

2. from pr1002: Within 7000 generations and population = 30, we will reach the number 925192 by spending 4 min and 37 seconds. we tried it several times and the number was between 900,000 to 1,000,000.

----------


We will see that our code outputs moderate results with acceptable speed. The disadvantage of the code is that it does not have a good exploration capability and therefore gets stuck in local optimizations.

A solution we can  use is memetics algorithm . MA considers the best option by defining a neighborhood for each crossover child and do local search in each neighborhood and replace it to the answer.

the rest is similar to GA

In [7]:
def Cross_Over(dad : Traverse, mom : Traverse, Cities:dict) -> Traverse:
    child = Traverse(dad.size,Cities)
    randidx = random.randint(2,mom.size+1)
    child.travel = mom.travel[:randidx]
    for i in range(randidx,dad.size):
        if dad.travel[i] not in child.travel:
            child.travel.append(dad.travel[i])
        else:
            j = i
            while dad.travel[j] in child.travel:
                j = mom.travel.index(dad.travel[j])
            child.travel.append(dad.travel[j])
    child.fitness = child.totalPath(Cities)
    
    #local Search:
    factor = child.fitness
    for i in range(1,child.size-2):
        for j in range(i+1,child.size-1):
            gamma = (factor - (child.path(child.travel[i-1],child.travel[i],Cities)+child.path(child.travel[j],child.travel[j+1],Cities))
                + (child.path(child.travel[i-1],child.travel[j],Cities)+child.path(child.travel[i],child.travel[j+1],Cities)))
            if gamma < factor:
                child.travel[i],child.travel[j] = child.travel[j],child.travel[i]
                factor = gamma
    child.fitness = child.totalPath(Cities)
    return child

In [8]:
def MA(iteration:int, Cities:dict, alpha:float, citycount:int, Population:int) -> Traverse:
    population = [Traverse(citycount,Cities) for _ in range(Population)]
    population.sort(key = lambda x : x.fitness)
    
    for _ in range(iteration):
        children = []
        for t in range(Population//4):
            parents = random.choices(population,k = 2,weights = [1/(gene+1) for gene in range(1,len(population)+1)])
            child = Cross_Over(parents[0],parents[1],Cities)
            child = Mutation(child,alpha,Cities)
            children.append(child)
        
        population += children
        population.sort(key = lambda x : x.fitness)
        population = population[:Population]
        
        print(_,'best_fitness:' , population[0].fitness)
    return population[0]

In [24]:
Cities = dict()
with open("gr229.txt",mode="r") as file:
    count = 0
    for line in file.readlines():
        line = str(line)
        line = line[:-1]
        line = line.split(" ")
        if count == 0:
            CityCount = int(line[0])
            count = 1
        else:
            Cities.update({int(line[0]):[float(line[1]),float(line[2])]})
best_ma = MA(20,Cities,0.5,CityCount,30)
print(best_ma.travel)

0 best_fitness: 1187.6315023649809
1 best_fitness: 1063.9708348942859
2 best_fitness: 1002.5906483308568
3 best_fitness: 1002.5906483308568
4 best_fitness: 991.4698649239127
5 best_fitness: 972.0632396658697
6 best_fitness: 972.0632396658697
7 best_fitness: 972.0632396658697
8 best_fitness: 966.9826673025512
9 best_fitness: 946.632146672542
10 best_fitness: 946.632146672542
11 best_fitness: 946.632146672542
12 best_fitness: 923.1897195167992
13 best_fitness: 923.1897195167992
14 best_fitness: 902.9347106426911
15 best_fitness: 902.9347106426911
16 best_fitness: 902.9347106426911
17 best_fitness: 893.6791866431187
18 best_fitness: 888.734893210337
19 best_fitness: 888.734893210337
[177, 174, 175, 176, 186, 185, 183, 182, 178, 180, 181, 179, 40, 39, 41, 46, 184, 50, 48, 47, 42, 45, 43, 44, 49, 188, 189, 190, 191, 193, 192, 194, 195, 196, 187, 197, 198, 199, 200, 223, 222, 202, 221, 220, 219, 216, 215, 214, 213, 209, 208, 207, 206, 210, 211, 203, 201, 158, 204, 212, 205, 150, 157, 155, 14

Result:

1. from gr229: Within 20 generations and population = 30, we will reach the number 883 by spending 12 seconds. we tried it several times and the number was between 800 to 950.

2. from pr1002: Within 10 generations and population = 20, we will reach the number 215,706 by spending 1 min and 29 seconds. we tried it several times and the number was between 180,000 to 250,000.

----------

Although the speed of the MA decreased due to the local search, we can see that it produces much better answers when compared to GA.